In [1]:
import keras
import re
import pandas as pd
import pymongo
import jieba
jieba.set_dictionary("dict.txt.big.txt")

Using TensorFlow backend.


In [2]:
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["semester"]
mycol = mydb["ettoday"]

In [3]:
K_train =  pd.read_csv("../train_p.csv")
corpus_x1 = K_train.title1_zh
corpus_x2 = K_train.title2_zh
corpus = pd.DataFrame()
corpus = pd.concat([
    corpus,corpus_x1, corpus_x2])
print(len(corpus))
corpus.columns = ["title"]
corpus.head()

641090


,title
0,2017養老保險又新增兩項，農村老人人人可申領，你領到了嗎
1,"""你不來深圳，早晚你兒子也要來""，不出10年深圳人均GDP將超香港"
2,"""你不來深圳，早晚你兒子也要來""，不出10年深圳人均GDP將超香港"
3,"""你不來深圳，早晚你兒子也要來""，不出10年深圳人均GDP將超香港"
4,"""用大蒜鑒別地溝油的方法,怎麼鑒別地溝油"


In [36]:
def cleanData(sentence):
    sentence = re.sub("\W+","",sentence)
    texts = jieba.cut(sentence)
    arr = []
    for t in texts:
        if(t.isdigit() or re.search("[^a-zA-Z]+",t) == None):
            arr.append(t)
            continue
        else:
            for tee in t:
                arr.append(tee)
    return " ".join(arr)

def stringify(text):
    return str(text)

In [5]:
corpus["title"] = corpus["title"].apply(cleanData)
corpus.head()

Building prefix dict from D:\program\semester_final\clean_data\dict.txt.big.txt ...
Loading model from cache C:\Users\PC-Henry\AppData\Local\Temp\jieba.u6179e01b4c6492e8310058a8757f6034.cache
Loading model cost 1.309 seconds.
Prefix dict has been built succesfully.


,title
0,2017 養 老 保 險 又 新 增 兩 項 農 村 老 人 人 人 可 申 領 你 領 到...
1,你 不 來 深 圳 早 晚 你 兒 子 也 要 來 不 出 10 年 深 圳 人 均 GDP...
2,你 不 來 深 圳 早 晚 你 兒 子 也 要 來 不 出 10 年 深 圳 人 均 GDP...
3,你 不 來 深 圳 早 晚 你 兒 子 也 要 來 不 出 10 年 深 圳 人 均 GDP...
4,用 大 蒜 鑒 別 地 溝 油 的 方 法 怎 麼 鑒 別 地 溝 油


In [7]:
ettoday = []
for news in mycol.find({},{"title":1,"_id":0}):
    ettoday.append(news["title"])
print(ettoday[:10])


['快訊／高嘉瑜晚間好奇找阿北求證\u3000柯P終於「一句話」解答了', '美眉的夏天／最HOT名單出爐\u3000她激戰搶下冠軍抱千元大獎', '搭西寧軍艦南偵\u300022名教師登太平島深化主權意識', '桃園區慶讚中元水燈排車繞境\u3000傳承百年特色慶典', '鳳飛飛故事展、電影展…等紀念活動\u3000緬懷永遠的國民天后', '韓國瑜選總統8月3日從桃園出發\u3000要讓全國看到藍軍團結力量', '規劃「個人服裝包」\u3000國軍新兵服裝用APP快速搞定', '嘉義「火燈夜巡、大士爺祭」深獲在地認同\u3000下一步推向國際', '台民黨「不分區名單」她第一個被點名！\u3000柯文哲傾向不選才組黨', '影／她吃藥睡死…起床驚見「手被蟑螂咬」模樣超慘\u3000網加碼曝：而且非常痛']


In [9]:
print(len(corpus))
ettoday = [cleanData(news) for news in ettoday]

641090


In [29]:
ettoday2 = pd.DataFrame(ettoday)
ettoday2.columns = ["title"]
ettoday2.head()

,title
0,快 訊 高 嘉 瑜 晚 間 好 奇 找 阿 北 求 證 柯 P 終 於 一 句 話 解 答 了
1,美 眉 的 夏 天 最 HOT 名 單 出 爐 她 激 戰 搶 下 冠 軍 抱 千 元 大 獎
2,搭 西 寧 軍 艦 南 偵 22 名 教 師 登 太 平 島 深 化 主 權 意 識
3,桃 園 區 慶 讚 中 元 水 燈 排 車 繞 境 傳 承 百 年 特 色 慶 典
4,鳳 飛 飛 故 事 展 電 影 展 等 紀 念 活 動 緬 懷 永 遠 的 國 民 天 后


In [34]:
copus = pd.concat([corpus,ettoday2])
copus.reset_index(drop=True)
print(len(copus))
copus.head()

690851


,title
0,2017 養 老 保 險 又 新 增 兩 項 農 村 老 人 人 人 可 申 領 你 領 到...
1,你 不 來 深 圳 早 晚 你 兒 子 也 要 來 不 出 10 年 深 圳 人 均 GDP...
2,你 不 來 深 圳 早 晚 你 兒 子 也 要 來 不 出 10 年 深 圳 人 均 GDP...
3,你 不 來 深 圳 早 晚 你 兒 子 也 要 來 不 出 10 年 深 圳 人 均 GDP...
4,用 大 蒜 鑒 別 地 溝 油 的 方 法 怎 麼 鑒 別 地 溝 油


In [43]:
fake = pd.read_csv("fake_clean2.csv",header=None)
fake.columns = ["title1","title2","label"]
unfake = pd.read_csv("unfake_clean.csv",header=None)
unfake.columns = ["title1","title2","label"]
unfake["title1"] = unfake["title1"].apply(stringify)
unfake = unfake[unfake["title1"]!='nan']
train_data = pd.concat([fake,unfake],axis=0)
print(len(train_data))
train_data.head()

4057


,title1,title2,label
0,看看香港-[黃之鋒]的惡行惡狀欺負老人家算什麼好漢？暴徒來台卻變成英雄貴賓把台灣道德價值摧毀...,【錯誤】網傳影片「看看香港，黃之鋒的惡行惡狀，充分沒有教養的港獨暴力份子，...欺負老人家算...,False
1,3、燒烤時戴隱形眼鏡遇高溫會使隱形眼鏡熔化而致盲,【錯誤】網傳「在烤肉...請將你的隱形眼鏡拿掉！一個21歲男生生火時，過熱的溫度熔化隱形眼鏡」？,False
2,剛出爐的麵包傷胃,《謠言終結站》剛出爐的麵包會致癌？ 專家斥無稽 - 生活 - 自由時報電子報,False
3,昨晚，香港一名參與示威女子被同伴擊中眼睛，可能致視力嚴重受損，他們竟還反污衊香港警方,早報：中國央視報導香港女子右眼受傷，引用假照片，「新聞」信源無可稽考｜端傳媒 Initium...,False
4,你還在吃龍眼嗎？看完後震驚了！一定要告訴家人啊！分享~：,【假養生】近視從800度降到200度？謠言讓醫生表示效果有限，不要過量了！ ｜MyGoPen,False


In [45]:
with open("./news_platform.txt") as f:
    news_platform = f.readlines()
news_del = "|".join([re.sub("\W+","",new) for new in news_platform])
with open("./other_platform.txt") as f:
    other_platform = f.readlines()
other_del = "|".join([re.sub("\W+","",new) for new in other_platform])
_del = news_del+"|"+other_del

def clean_mark(text):
    return re.sub(_del,"",text)

In [48]:
train = pd.DataFrame()
train["title"] = pd.concat([train_data["title1"].apply(clean_mark).apply(cleanData),\
                            train_data["title2"].apply(clean_mark).apply(cleanData)])
# train["title1"] = train_data["title1"].apply(clean_mark)
# train["title2"] = train_data["title2"].apply(clean_mark)
# train["label"] = train_data["label"]
print(len(train))
train.head()

8114


,title
0,看 看 香 港 黃 之 鋒 的 惡 行 惡 狀 欺 負 老 人 家 算 什 麼 好 漢 暴 ...
1,3 燒 烤 時 戴 隱 形 眼 鏡 遇 高 溫 會 使 隱 形 眼 鏡 熔 化 而 致 盲
2,剛 出 爐 的 麵 包 傷 胃
3,昨 晚 香 港 一 名 參 與 示 威 女 子 被 同 伴 擊 中 眼 睛 可 能 致 視 ...
4,你 還 在 吃 龍 眼 嗎 看 完 後 震 驚 了 一 定 要 告 訴 家 人 啊 分 享


In [49]:
c = pd.concat([copus,train])
print(len(c))
c.head()

698965


,title
0,2017 養 老 保 險 又 新 增 兩 項 農 村 老 人 人 人 可 申 領 你 領 到...
1,你 不 來 深 圳 早 晚 你 兒 子 也 要 來 不 出 10 年 深 圳 人 均 GDP...
2,你 不 來 深 圳 早 晚 你 兒 子 也 要 來 不 出 10 年 深 圳 人 均 GDP...
3,你 不 來 深 圳 早 晚 你 兒 子 也 要 來 不 出 10 年 深 圳 人 均 GDP...
4,用 大 蒜 鑒 別 地 溝 油 的 方 法 怎 麼 鑒 別 地 溝 油


In [50]:
print(c.shape)

(698965, 1)


In [86]:
tokenizer = keras \
    .preprocessing \
    .text \
    .Tokenizer(12000)

In [66]:
c = c[c["title"].str.isnumeric() == False]
print(c.shape)
c_a = c.values.tolist()
print(len(c_a))
c_a = [i[0] for i in c_a]
print(len(c_a))

(698963, 1)
698963
698963


In [87]:
tokenizer.fit_on_texts(c_a)
print(tokenizer.num_words)

12000


In [88]:
print(tokenizer.index_word.get(200))

得


In [89]:
import pickle

# 應該將所有資料集整合統一成一個 pickle
with open("tokenized2.pickle","wb") as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)